In [86]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas

# import modules
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import datetime
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet') 
nltk.download('stop') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading stopord: Package 'stopord' not found in
[nltk_data]     index


False

In [2]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Location:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")
    print(f"Date:{ith_tweet[9]}")

In [3]:
#importer dansk som sprog
#pip install afinn
from afinn import Afinn
afinn = Afinn(language='da')
afinn.score('Hvis ikke det er det mest afskyelige flueknepperi...')
-6.0

#user = api.get_status(id)
  
# fetching the created_at attribute
#created_at = status.created_at 

#today = datetime.date.today()

-6.0

In [4]:
# function to perform data extraction
def scrape(words, date_since, numtweet):
      
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags', 'date'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="da",
                           since=date_since, tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        date = tweet.created_at
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, following,
                     followers, totaltweets, retweetcount, text, hashtext, date]
        db.loc[len(db)] = ith_tweet
        
          
        # Function call to print tweet data on screen
        printtweetdata(i, ith_tweet)
        i = i+1
    filename = 'df3.csv'
      
    # we will save our database as a CSV file.
    db.to_csv(filename)
  
  
if __name__ == '__main__':
      
    # Enter your own credentials obtained 
    # from your developer account
    consumer_key = "HlgemqHuJ83L0NkQbg4rZQLQ1"
    consumer_secret = "iT2MPE5T2n4pGqDlTbFsStnlPyRPgOvk64bEhdRA7ZTE86G60g"
    access_key = "1427158064092876800-MNinx9VxTMhQf07xIutt2upLZvAaPn"
    access_secret = "Utlu32drsCwKj3SnCAYU2qKZ66VNovYBxLxdRu4DeDWrA"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
      
    # Enter Hashtag and initial date
    print("Enter Twitter HashTag to search for")
    words = input()
    print("Enter Date since The Tweets are required in yyyy-mm--dd")
    date_since = input()
      
    # number of tweets you want to extract in one run
    numtweet = 10000
    scrape(words, date_since, numtweet)
    print('Scraping has completed!')

Enter Twitter HashTag to search for


KeyboardInterrupt: Interrupted by user

### Combining dataframes

In [6]:
df1=pd.read_csv('df1.csv')
df2=pd.read_csv('df2.csv')
df3=pd.read_csv('df3.csv')

obs = [df1, df2, df3]

data_set = pd.concat(obs)
data_set = data_set.reset_index(drop = True)

,Unnamed: 0,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags,date
0,0,SupAyaLaya,NaN,NaN,1228,521,34115,1,"Beder alle indberette dette tweet, da det tyde...",[],2021-08-16 23:59:33
1,1,Mullarrd,"Nearsighted scrap pile, perpetual lefty. Searc...",Pangaea,515,536,39951,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 21:14:11
2,2,GitteKJP,"Søger viden, nyheder og indsigter til eftertanke.",København,972,947,231239,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 16:41:45
3,3,idamariehansen7,NaN,Nykøbing Sjælland,1292,346,21549,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 16:07:07
4,4,ThomasHeie,Fængselsbetjent - FREM'er - Fotograf - FDFer,"København, Danmark",111,269,21567,0,Indsatsen i Afghanistan har kostet Danmark 15 ...,"['dkpol', 'COVID19dk']",2021-08-16 15:48:37
...,...,...,...,...,...,...,...,...,...,...,...
1731,26,KlausKblog,"Klaus K, journalist & sangskriver i Danser med...",København,511,2464,33254,28,Israels massevaccination: \n25% stigning i hje...,[],2021-08-11 18:48:21
1732,27,HettingClaus,"""Three things remain not long hidden: The Sun,...",Denmark,2249,2717,15616,28,Israels massevaccination: \n25% stigning i hje...,[],2021-08-11 17:59:14
1733,28,nbm_1981,IT Automation Consultant (MCAAA/MCSA/MCP). Lik...,"København, Danmark",612,624,12430,28,Israels massevaccination: \n25% stigning i hje...,"['dkpol', 'dkmedier', 'coronadk', 'covid19dk',...",2021-08-11 17:50:59
1734,29,transceiverfreq,xi=aixi'-(d+ai0+ai1)xi+rxi(fi-Φ ) | Freelance ...,"Frederiksberg, Denmark",339,1089,44893,0,@DRNyheder @tv2newsdk @moderna_tx @pfizer Jeg ...,"['dkpol', 'dkmedier', 'COVID19']",2021-08-10 19:29:12


### Cleaning the DataFrame

In [ ]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('danish'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [29]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [49]:
with open('stopord.txt', 'r') as f:
    stopwords_danish = []
    for line in f.

[nltk_data] Error loading stopord: Package 'stopord' not found in
[nltk_data]     index


False

In [85]:
#Specifying text column
sentence = data_set1.text.values[0]

sent = nltk.tokenize.word_tokenize(sentence)

emoji = emoji_pattern.sub(r'', str(sent))

text = emoji.lower()

stop_words_list = nltk.corpus.stopwords.words("danish")
sent_sw_removed = [i for i in nltk.word_tokenize(text) if i not in stop_words_list]
sent_sw_removed


['[',
 "'beder",
 "'",
 ',',
 "'alle",
 "'",
 ',',
 "'indberette",
 "'",
 ',',
 "'dette",
 "'",
 ',',
 "'tweet",
 "'",
 ',',
 "'",
 ',',
 "'",
 ',',
 "'da",
 "'",
 ',',
 "'det",
 "'",
 ',',
 "'tydeligt",
 "'",
 ',',
 "'opfordrer",
 "'",
 ',',
 "'til",
 "'",
 ',',
 "'overgreb",
 "'",
 ',',
 "'",
 ',',
 "'",
 ',',
 "'tvang",
 "'",
 ',',
 "'og",
 "'",
 ',',
 "'vold",
 "'",
 ',',
 "'mod",
 "'",
 ',',
 "'børn",
 "'",
 ',',
 "'og",
 "'",
 ',',
 "'unge",
 "'",
 ',',
 "'",
 '.',
 "'",
 ',',
 "'bør",
 "'",
 ',',
 "'ikke",
 "'",
 ',',
 "'høre",
 "'",
 ',',
 "'hjemme",
 "'",
 ',',
 "'på",
 "'",
 ',',
 "'sociale",
 "'",
 ',',
 "'medier",
 "'",
 ',',
 "'",
 '#',
 "'",
 ',',
 "'coronadk",
 "'",
 ',',
 "'",
 '#',
 "'",
 ',',
 "'dkmedier",
 "'",
 ',',
 "'",
 '#',
 "'",
 ',',
 "'dkpol",
 "'",
 ',',
 "'",
 '#',
 "'",
 ',',
 "'covid19dk",
 "'",
 ',',
 "'",
 '@',
 "'",
 ',',
 "'sstbrostrom",
 "'",
 ',',
 "'",
 '@',
 "'",
 ',',
 "'heunicke",
 "'",
 ',',
 "'hvad",
 "'",
 ',',
 "'er",
 "'",
 ',',
 "'det",
 "

In [61]:
#Dropping collumns
data_set1 = data_set.drop(['Unnamed: 0','username', 'description', 'followers', 'following'], axis=1).reset_index(drop = True)

#Specifying text column
sentence = data_set1.text.values

#Filtering out emojis etc. using the function
#tweet = emoji_pattern.sub(r'', str(sentence))

#Tokenizing string objects

#lwr_tweet = [nltk.word_tokenize(i.lower()) for i in data_set1.text.values]

#Dropping stopwords

#stop_words_list = nltk.corpus.stopwords.words("danish")
#sent_sw_removed = [i for i in tweet if i not in stop_words_list]
#sent_sw_removed
